In [ ]:
import geopandas as gpd
import osmnx as ox
from IPython.display import display
import numpy as np
from tqdm.auto import tqdm
import pandas as pd

from srai.embedders import Highway2VecEmbedder
from srai.loaders import osm_way_loader

## Basic usage

In [ ]:
# embedder = Highway2VecEmbedder()

In [ ]:
gdf_place = ox.geocode_to_gdf("Wrocław, Poland")
gdf_place.plot()

In [ ]:
from srai.loaders import OSMWayLoader, NetworkType

osmwl = OSMWayLoader(NetworkType.DRIVE)
gdf_nodes, gdf_edges = osmwl.load(gdf_place)

In [ ]:
ax = gdf_edges.plot(linewidth=1, figsize=(12, 7))
gdf_nodes.plot(ax=ax, markersize=3, color="red")

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data.dataloader import DataLoader
from sklearn.model_selection import train_test_split

display(torch.cuda.is_available())


class LitAutoEncoder(pl.LightningModule):
    def __init__(self, in_dim: int, hidden_dim: int = 64, latent_dim: int = 3, lr: float = 1e-3):
        super().__init__()

        self.save_hyperparameters()

        self.encoder = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, in_dim),
        )
        self.lr = lr

    def forward(self, x):
        z = self.encoder(x)
        return z

    def training_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, "train")

    def validation_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, "val")

    def test_step(self, batch, batch_idx):
        return self._common_step(batch, batch_idx, "test")

    def _prepare_batch(self, batch, batch_idx):
        x = batch
        # x = x.view(x.size(0), -1)
        return x

    def _common_step(self, batch, batch_idx, stage: str) -> torch.Tensor:
        x = self._prepare_batch(batch, batch_idx)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        # loss = F.binary_cross_entropy_with_logits(x_hat, x)

        self.log(f"{stage}_loss", loss, on_epoch=True, on_step=True, prog_bar=True)

        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.lr)
        return optimizer

In [ ]:
gdf_edges

In [ ]:
gdf_edges_features = gdf_edges.drop(
    columns=["osmid", "name", "reversed", "length", "geometry", "from", "to", "ref"]
)
gdf_edges_features

In [ ]:
test_size = 0.2
random_seed = 42
batch_size = 128
num_workers = 6
shuffle = True
hidden_dim = 64
enc_out_dim = 40
latent_dim = 30
epochs = 10
kl_coeff = 0.1
lr = 1e-3
n_features = gdf_edges_features.shape[1]

pl.seed_everything(random_seed, workers=True)

X = torch.Tensor(gdf_edges_features.values)
X_train, X_test = train_test_split(X, test_size=test_size, random_state=random_seed, shuffle=True)
X_train_dl = DataLoader(
    X_train, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=True
)
X_test_dl = DataLoader(
    X_test, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True
)

model = LitAutoEncoder(in_dim=n_features, hidden_dim=hidden_dim, latent_dim=latent_dim, lr=lr)

# logger_tb = pl.loggers.TensorBoardLogger("tb_logs", name="test_model")

trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=epochs)
trainer.fit(model, train_dataloaders=X_train_dl, val_dataloaders=X_test_dl)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/